
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/EffiSciencesResearch/hackathon42/blob/main/starting_pack_hackathon42.ipynb)


In [16]:
#@title Télécharger des données qui sont sur Github 

# Il faudrait installer git large file system avant de cloner
!git lfs install
!git clone https://github.com/EffiSciencesResearch/hackathon42.git

Updated git hooks.
Git LFS initialized.
Cloning into 'hackathon42'...
remote: Enumerating objects: 626, done.
remote: Counting objects: 100% (89/89), done.
remote: Compressing objects: 100% (78/78), done.
remote: Total 626 (delta 15), reused 80 (delta 10), pack-reused 537
Receiving objects: 100% (626/626), 2.40 MiB | 16.73 MiB/s, done.
Resolving deltas: 100% (124/124), done.
tcmalloc: large alloc 1471086592 bytes == 0x55cad749a000 @  0x7f053af6e2a4 0x55ca9b19511f 0x55ca9b17225b 0x55ca9b126f33 0x55ca9b0cb22a 0x55ca9b0cb6e6 0x55ca9b0e8451 0x55ca9b0e89e9 0x55ca9b0e8f13 0x55ca9b18de12 0x55ca9b02f162 0x55ca9b015a65 0x55ca9b016725 0x55ca9b01572a 0x7f053a2b5c87 0x55ca9b01577a
tcmalloc: large alloc 1471086592 bytes == 0x55cad749a000 @  0x7f053af6e2a4 0x55ca9b19511f 0x55ca9b17225b 0x55ca9b126f33 0x55ca9b0cb22a 0x55ca9b0cb6e6 0x55ca9b0e8451 0x55ca9b0e89e9 0x55ca9b0e8f13 0x55ca9b18de12 0x55ca9b02f162 0x55ca9b015a65 0x55ca9b016725 0x55ca9b01572a 0x7f053a2b5c87 0x55ca9b01577a
Filtering content: 100

In [22]:
#@title On choisi le dataset sur lequel on travaiile
# Création de traing dataset et test dataset

%cd /content/hackathon42

import numpy as np
from tqdm.notebook import tqdm
import os
from os.path import join
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.nn.functional as F

use_cuda = False
device = torch.device("cuda" if use_cuda else "cpu")

datasets_names = sorted(os.listdir(path='datasets'), key=lambda name: int(name[:2]))
datasets_names = {int(datasets_name.split("_")[0]): datasets_name for datasets_name in datasets_names}
print(datasets_names)

# You can choose the dataset here by modifying the brackets
dataset_name = join('datasets', datasets_names[3])
# assert "01_mnist_cc" not in dataset_name, "This colab is destined for MNIST, and not for the other datasets. The code needs to be adapted :) "
print(dataset_name)

X_labeled = np.load(join(dataset_name, "X_labeled.npy"))
y_labeled = np.load(join(dataset_name, "y_labeled.npy"))
X_unlabeled = np.load(join(dataset_name, "X_unlabeled.npy"))
X_val = np.load(join(dataset_name, "X_val.npy"))


X_train, X_test, y_train, y_test = train_test_split(X_labeled, y_labeled)


X_train = torch.tensor(X_train, device=device).float()
X_test = torch.tensor(X_test, device=device).float()
y_train = torch.tensor(y_train, device=device)
y_test = torch.tensor(y_test, device=device)

# On s'intéresse à quoi cela resemble X_val
# for i in range(10):
#     plt.imshow(X_val[i, 0])
#     plt.show()
#     print(i)
#     print("")

/content/hackathon42
{0: '00_toy_dataset', 1: '01_mnist_cc', 2: '02_mnist_constant_image', 3: '03_mnist_constant_image_random_row', 4: '04_mnist_uniform_color_random_row', 5: '05_mnist_uniform_color_low_mix_rate', 6: '06_mnist_sum', 8: '08_mnist_sum_noise_level', 12: '12_mnist_mysterious', 13: '13_mnist_mysterious', 23: '23_mnist_embed', 456: '456_mnist_embed', 888: '888_vehicle_animal', 999: '999_humans'}
datasets/03_mnist_constant_image_random_row


In [23]:
#@title Création de Network (CNN)
# On définit les exécutions pour chaque couche de CNN

%pip install sewar
from sewar.full_ref import rmse

class LeNet(nn.Module):
    def __init__(self, n_classes=10, trois= X_val[0, 0][:, 28:], zero =  X_val[4, 0][:, :28]):
        super().__init__()       
        
        self.conv1 = nn.Conv2d(1, 16, 5)
        self.conv2 = nn.Conv2d(16, 32, 5)
        self.fc1 = nn.Linear(32*11*4, 100)
        self.fc2 = nn.Linear(100, n_classes)
        self.flat = nn.Flatten()
        self.trois = trois
        self.zero = zero

    # On définit les exécutions pour chaque kernel et chaque channel        
    def forward(self, x):
        mask = np.concatenate((np.zeros((28, 28)), np.ones((28, 28))), axis=1)
        mask = torch.tensor([[mask]], device=device).int() 
        out = torch.masked_fill(x, mask, value=0 )
        out = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        out = F.max_pool2d(F.relu(self.conv2(out)), 2)
        out = self.flat(out)
        out = F.relu(self.fc1(out))
        x = F.dropout(x, training=self.training)
        pred = self.fc2(out)
        
        return pred
        
net = LeNet().float()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [24]:
#@title Entraînement et test 

# Epoch, le nombre du fois qu'il appprend, ceci n'est pas pareil que le nombre du fois de l'itération for
# criterion, loss function

import pandas as pd

tbatch = 100
nb_epochs = 20

nbbatchs = int(X_train.shape[0]/tbatch)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=0.01)

net.train()

def evaluate(X_test):
    net.eval()
    with torch.no_grad():
        y_pred = net(X_test)
    correct_test = (torch.argmax(y_pred, axis=1) == y_test).sum().item()
    net.train()
    return correct_test/len(X_test)

# Itération sur le dataset
for epoch in tqdm(range(nb_epochs)):  
    correct = 0
    for i in (range(nbbatchs)):
        
        optimizer.zero_grad()
        
        X_batch = X_train[i*tbatch:(i+1)*tbatch]
        y_batch = y_train[i*tbatch:(i+1)*tbatch]

        # Compute forward / backward
        y_pred = net(X_batch)
        correct += (torch.argmax(y_pred, axis=1) == y_batch).sum().item()

        loss = criterion(y_pred, y_batch)
        

        loss.backward()
        optimizer.step()
        # break
    print("acc train:", correct/X_train.shape[0], "acc test:", evaluate(X_test))

  0%|          | 0/20 [00:00<?, ?it/s]

acc train: 0.43473451327433627 acc test: 0.4966887417218543
acc train: 0.6714601769911505 acc test: 1.0
acc train: 0.995575221238938 acc test: 1.0
acc train: 0.995575221238938 acc test: 1.0
acc train: 0.995575221238938 acc test: 1.0
acc train: 0.995575221238938 acc test: 1.0
acc train: 0.995575221238938 acc test: 1.0
acc train: 0.995575221238938 acc test: 1.0
acc train: 0.995575221238938 acc test: 1.0
acc train: 0.995575221238938 acc test: 1.0
acc train: 0.995575221238938 acc test: 1.0
acc train: 0.995575221238938 acc test: 1.0
acc train: 0.995575221238938 acc test: 1.0
acc train: 0.995575221238938 acc test: 1.0
acc train: 0.995575221238938 acc test: 1.0
acc train: 0.995575221238938 acc test: 1.0
acc train: 0.995575221238938 acc test: 1.0
acc train: 0.995575221238938 acc test: 1.0
acc train: 0.995575221238938 acc test: 1.0
acc train: 0.995575221238938 acc test: 1.0


In [ ]:
#@title Traitement aux images afin d'enlever une ambiguité
import matplotlib.pyplot as plt

# for i in range(10):
#     plt.imshow(X_val[i, 0])
#     plt.show()
#     print(i)
#     print("")

# On remarque que dans les images de X_val, pour chaque X[i, 0] il y a toujours soit une image de 0 soit une image de 3 qui se répète
# Par exemple, 0 à droite de X_val[0, 0] et 3 à droite de X_val[4, 0]
trois = X_val[0, 0][:, 28:]
zero = X_val[4, 0][:, :28]

# On enlève donc l'image de soit 0 soit 3 qui se répète, et duplique l'autre image afin d'avoir une mailleure précision
for i in range(21):
    x1 = X_val[i, 0][:, :28]
    x2 = X_val[i, 0][:, 28:]
    if rmse(x1,trois) < 0.1 or rmse(x1,zero) < 0.1 :
       mask = np.concatenate((x2,x2), axis=1)
    else :
       mask = np.concatenate((x1,x1), axis=1)
       
    mask = torch.tensor(mask, device=device)
    x = torch.tensor(mask, device=device).float()
    plt.imshow(x)
    plt.show()
    print(i)

In [ ]:
#@title Visualiser les deux images qu'on doit enlever de chaque paire

trois = X_val[11, 0][:, :28]
zero= X_val[9, 0][:, :28]
plt.imshow(trois)
plt.show()
plt.imshow(zero)
plt.show()

In [31]:
#@title Prédire les labels de X_val
# On pourra demander maximum 5 labels de X_val en utilisant API(le bloc de code tout en bas) pour les comparer
# Si le résultat semble bon on pourra implenter le code

X_target = torch.tensor(X_val, device=device).float()
# net.eval()
with torch.no_grad():
  y_target = net(X_target)
    
pred_target = torch.argmax(y_target, dim=1)
print(pred_target[0:21])

tensor([3, 3, 0, 3, 3, 3, 3, 0, 3, 0, 3, 3, 3, 3, 0, 3, 0, 0, 0, 3, 3])


In [32]:
#@title Sauvegarder le résultat en tant que csv file

print(pred_target.shape)
pred_target = pred_target.numpy()
df = pd.DataFrame(pred_target)
df.to_csv("submission_02.csv", header=False, index=False)

torch.Size([1206])


In [33]:
#@title Envoyer une demande afin de récuperer le label d'un X_val

import requests
import json
 
res = requests.post("https://leaderboard42.herokuapp.com/reveal/", data={
        'username': 'SkynetV2.0',
        'password': "paristoulouse",
        'exercise_id': 4,
        'datum_id': 7
   })
 
try:
   res = json.loads(res.content)
   print(res)
except:
    print("Error")
    print(res.content)

# Cela retourne la réponse comme ceci
# {'exercise_id': 3, 'datum_id': 456, 'label': 0, 'previously revealed': [12]}

Error
b"\n\n\nUsage examples:\n $ curl -F username=awesome_team -F password=secret_password -F exercise_id=3 -F datum_id=456 https://leaderboard42.herokuapp.com/reveal/\n\n >>> import requests\n >>> requests.post(https://leaderboard42.herokuapp.com/reveal/, data={\n         'username': 'awesome_team',\n         'password': 'secret_password',\n         'exercise_id': 3,\n         'datum_id': 456\n     })\n"
